In [1]:
from PIL import Image
def smooth_gif_resize(gif, frameWidth, frameHeight):
    gif = Image.open(gif)
    gifWidth0, gifHeight0 = gif.size

    widthRatio = frameWidth / gifWidth0
    heightRatio = frameHeight / gifHeight0

    if widthRatio >= heightRatio:
        gifWidth1 = gifWidth0 * heightRatio
        gifHeight1 = frameHeight
        return gifWidth1, gifHeight1

    gifWidth1 = frameWidth
    gifHeight1 = gifHeight0 * widthRatio
    return round(gifWidth1), round(gifHeight1)

In [2]:
from PyQt5 import uic
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QAction, QFileDialog, QWidget, QPushButton, QVBoxLayout
from PyQt5.QtGui import QMovie
from PyQt5 import QtCore
from PyQt5.QtCore import QSize
import mediapipe_helper as mph
import mixamo_helper as mmh
import json
import pyglm_helper as glmh
import ntpath

#UI파일 연결
#단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
form_class = uic.loadUiType("gui.xml")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.statusBar().showMessage('Ready')

        self.setFixedWidth(self.frameGeometry().width())
        self.setFixedHeight(self.frameGeometry().height())

        self.btn_add_model.adjustSize()
        #버튼에 기능을 연결하는 코드
        self.btn_add_model.clicked.connect(self.add_model_path)
        self.btn_add_gif.clicked.connect(self.add_gif_path)
        self.btn_add_output.clicked.connect(self.add_output_path)
        self.btn_convert.clicked.connect(self.convert)
        self.cmb_gif.currentIndexChanged.connect(self.show_gif)
        self.h_slider_min_visibility.valueChanged.connect(self.set_lbl_min_visibility)
        self.h_slider_model_complexity.valueChanged.connect(self.set_lbl_model_complexity)
    
    def set_lbl_min_visibility(self):
        self.lbl_slider_min_visibility.setText(str(self.h_slider_min_visibility.value()))
    
    
    def set_lbl_model_complexity(self):
        self.lbl_slider_model_complexity.setText(str(self.h_slider_model_complexity.value()))

    def add_model_path(self):
        self.add_cmb_item_from_dialog("Add Model Json File", "./", "Json (*.json)", self.cmb_model)

    def add_gif_path(self):
        self.add_cmb_item_from_dialog("Add Animation GIF File", "./", "GIF (*.gif)", self.cmb_gif)

    def add_output_path(self):
        self.add_cmb_item_from_dialog("Select Output File", "./", "Json (*.json)", self.cmb_output, is_save=True)
   
    def convert(self):
        self.statusBar().showMessage('Converting...')
        if self.cmb_model.currentIndex() == -1 or  self.cmb_gif.currentIndex() == -1 or  self.cmb_output.currentIndex() == -1:
            self.statusBar().showMessage('Please try again')
            return
        try:
            model_path = self.cmb_model.currentText()
            gif_path = self.cmb_gif.currentText()
            output_path = self.cmb_output.currentText()
            mediapipe_json_object = mph.gif_to_mediapipe_json(fileName=gif_path, maxFrameNum=5000, modelComplexity=1)
            mixamo_json_object = mmh.mediapipeToMixamo(mph.get_name_idx_map(), mediapipe_json_object)
            anim_json = glmh.get_anim_json3(mixamo_json_object, model_path, is_hips_move= True, min_visibility= 0.6)
            with open(output_path, 'w') as f:
                json.dump(anim_json, f, indent= 2)
            self.statusBar().showMessage('Success!')
        except:
            self.statusBar().showMessage('Error!')
            return

    def add_cmb_item_from_dialog(self, title, path, filter, cmb, is_save = False):
        fname = self.show_dialog(title, path, filter, is_save)
        if fname != '':
            cmb.addItem(fname) 
            cmb.setCurrentIndex(cmb.count() - 1)

    # filter: "Images (*.png *.xpm .jpg);;Text files (.txt);;XML files (*.xml)"
    def show_dialog(self, title, path, filter, is_save = False):
        if not is_save:
            fname = QFileDialog.getOpenFileName(self, title, path, filter)
        else: 
            fname = QFileDialog.getSaveFileName(self, title, path, filter)
        return fname[0]

    def show_gif(self):
        if self.cmb_gif.currentIndex() != -1:
            movie = QMovie(self.cmb_gif.currentText())
            
            gif_size = QSize(*smooth_gif_resize(self.cmb_gif.currentText(), 150, 150))
            movie.setScaledSize(gif_size)
            self.lbl_gif_movie.setMovie(movie)
            self.lbl_gif_movie.adjustSize()
            movie.start()
app = QApplication(sys.argv)
myWindow = WindowClass() 
myWindow.show()
app.exec_()

0